In [25]:
# Virtual display
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

FileNotFoundError: [WinError 2] 系统找不到指定的文件。

In [1]:
import gymnasium

from huggingface_sb3 import load_from_hub, package_to_hub
from huggingface_hub import notebook_login # To log to our Hugging Face account to be able to upload models to the Hub.

from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor

In [2]:
import gymnasium as gym

# First, we create our environment called LunarLander-v2
env = gym.make("LunarLander-v3")

# Then we reset this environment
observation, info = env.reset()

for _ in range(100):
  # Take a random action
  action = env.action_space.sample()
  print("Action taken:", action)

  # Do this action in the environment and get
  # next_state, reward, terminated, truncated and info
  observation, reward, terminated, truncated, info = env.step(action)

  # If the game is terminated (in our case we land, crashed) or truncated (timeout)
  if terminated or truncated:
      # Reset the environment
      print("Environment is reset")
      observation, info = env.reset()

env.close()

Action taken: 3
Action taken: 1
Action taken: 2
Action taken: 2
Action taken: 2
Action taken: 0
Action taken: 0
Action taken: 2
Action taken: 1
Action taken: 1
Action taken: 1
Action taken: 0
Action taken: 0
Action taken: 2
Action taken: 1
Action taken: 3
Action taken: 1
Action taken: 2
Action taken: 3
Action taken: 2
Action taken: 0
Action taken: 3
Action taken: 0
Action taken: 0
Action taken: 2
Action taken: 1
Action taken: 3
Action taken: 2
Action taken: 0
Action taken: 3
Action taken: 1
Action taken: 0
Action taken: 3
Action taken: 3
Action taken: 1
Action taken: 2
Action taken: 0
Action taken: 1
Action taken: 1
Action taken: 2
Action taken: 3
Action taken: 2
Action taken: 1
Action taken: 2
Action taken: 3
Action taken: 2
Action taken: 2
Action taken: 1
Action taken: 2
Action taken: 2
Action taken: 2
Action taken: 2
Action taken: 2
Action taken: 0
Action taken: 1
Action taken: 2
Action taken: 0
Action taken: 1
Action taken: 1
Action taken: 0
Action taken: 3
Action taken: 1
Action t

In [3]:
# We create our environment with gym.make("<name_of_the_environment>")
env = gym.make("LunarLander-v3")
env.reset()
print("_____OBSERVATION SPACE_____ \n")
print("Observation Space Shape", env.observation_space.shape)
print("Sample observation", env.observation_space.sample()) # Get a random observation

_____OBSERVATION SPACE_____ 

Observation Space Shape (8,)
Sample observation [ 1.7471738   2.426423   -0.74409187  5.057401   -2.8202467   0.430484
  0.38746697  0.4501016 ]


In [4]:
print("\n _____ACTION SPACE_____ \n")
print("Action Space Shape", env.action_space.n)
print("Action Space Sample", env.action_space.sample()) # Take a random action


 _____ACTION SPACE_____ 

Action Space Shape 4
Action Space Sample 1


In [5]:
# Create the environment
env = make_vec_env('LunarLander-v3', n_envs=16)

In [6]:
# SOLUTION
# We added some parameters to accelerate the training
model = PPO(
    policy = 'MlpPolicy',
    env = env,
    n_steps = 1024,
    batch_size = 64,
    n_epochs = 4,
    gamma = 0.999,
    gae_lambda = 0.98,
    ent_coef = 0.01,
    verbose=1,
    device="cpu")

Using cpu device


In [22]:
# SOLUTION
# Train it for 1,000,000 timesteps
model.learn(
    total_timesteps=1000000, 
    progress_bar=True)
# Save the model
model_name = "ppo-LunarLander-v3"
model.save(model_name)

Output()

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 537      |
|    ep_rew_mean     | -1.21    |
| time/              |          |
|    fps             | 1725     |
|    iterations      | 1        |
|    time_elapsed    | 9        |
|    total_timesteps | 16384    |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 559          |
|    ep_rew_mean          | 3.54         |
| time/                   |              |
|    fps                  | 1351         |
|    iterations           | 2            |
|    time_elapsed         | 24           |
|    total_timesteps      | 32768        |
| train/                  |              |
|    approx_kl            | 0.0046173674 |
|    clip_fraction        | 0.0269       |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.2         |
|    explained_variance   | 0.786        |
|    learning_r

In [7]:
# Evaluate Model
model = PPO.load(
    path="./ppo-LunarLander-v3.zip",
    device="cpu")
eval_env = Monitor(gym.make("LunarLander-v3", render_mode='rgb_array'))
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=10, deterministic=True)
print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")

mean_reward=268.48 +/- 21.825398450637994


In [9]:
from huggingface_hub import login

login()

In [11]:
env_id = "LunarLander-v3"
repo_id = "sh10f/ppo-LunarLander-v3"

package_to_hub(model=model, 
               model_name="ppo-LunarLander-v3",
               model_architecture="PPO",
               env_id=env_id,
               eval_env=eval_env,
               repo_id=repo_id,
               commit_message="Test commit")

ℹ This function will save, evaluate, generate a video of your agent,
create a model card and push everything to the hub. It might take up to 1min.
This is a work in progress: if you encounter a bug, please open an issue.
Saving video to C:\Users\Lenovo\AppData\Local\Temp\tmpe6thmt2g\-step-0-to-step-1000.mp4
MoviePy - Building video C:\Users\Lenovo\AppData\Local\Temp\tmpe6thmt2g\-step-0-to-step-1000.mp4.
MoviePy - Writing video C:\Users\Lenovo\AppData\Local\Temp\tmpe6thmt2g\-step-0-to-step-1000.mp4



MoviePy - Done !
MoviePy - video ready C:\Users\Lenovo\AppData\Local\Temp\tmpe6thmt2g\-step-0-to-step-1000.mp4
✘ 'DummyVecEnv' object has no attribute 'video_recorder'
✘ We are unable to generate a replay of your agent, the package_to_hub
process continues
✘ Please open an issue at
https://github.com/huggingface/huggingface_sb3/issues
ℹ Pushing repo sh10f/ppo-LunarLander-v3 to the Hugging Face Hub


policy.pth:   0%|          | 0.00/43.6k [00:00<?, ?B/s]

policy.optimizer.pth:   0%|          | 0.00/88.0k [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

pytorch_variables.pth:   0%|          | 0.00/864 [00:00<?, ?B/s]

ppo-LunarLander-v3.zip:   0%|          | 0.00/151k [00:00<?, ?B/s]

ℹ Your model is pushed to the Hub. You can view your model here:
https://huggingface.co/sh10f/ppo-LunarLander-v3/tree/main/


CommitInfo(commit_url='https://huggingface.co/sh10f/ppo-LunarLander-v3/commit/6bf184b800bd31b95e32988992c6844757318f34', commit_message='Test commit', commit_description='', oid='6bf184b800bd31b95e32988992c6844757318f34', pr_url=None, repo_url=RepoUrl('https://huggingface.co/sh10f/ppo-LunarLander-v3', endpoint='https://huggingface.co', repo_type='model', repo_id='sh10f/ppo-LunarLander-v3'), pr_revision=None, pr_num=None)